In [ ]:
class Layer:
    def __init__(self, n_input_dims, n_neurons, activation=None, weights=None, bias=None):
        self.weights = weights if weights is not None else np.random.randn(
            n_input_dims, n_neurons) * np.sqrt(2 / n_input_dims)
        self.bias = bias if bias is not None else np.zeros(n_neurons)
        self.activation = activation
        self.A = None
        self.dA = None
        self.dZ = None
        self.dW = None
        self.db = None
        self.vdW = np.zeros_like(self.weights)
        self.vdb = np.zeros_like(self.bias)
        self.sdW = np.zeros_like(self.weights)
        self.sdb = np.zeros_like(self.bias)

    def activate(self, X):
        Z = np.dot(X, self.weights) + self.bias
        # 激活函数的输出
        self.A = self._apply_activation(Z)
        return self.A
    
    def _apply_activation(self, Z):
        # 激活函数
        if self.activation is None:
            return Z
        elif self.activation == 'relu':
            return np.maximum(0, Z)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-Z))
        elif self.activation == 'tanh':
            return np.tanh(Z)
        else:
            return Z
    
    def derivative(self, A):
        # 激活函数的导数
        if self.activation is None:
            return np.ones_like(A)
        elif self.activation == 'relu':
            # return np.where(A <= 0, 0., 1.)
            d = np.array(A, copy=True)
            d[A <= 0] = 0.
            d[A > 0] = 1.
            return d 
        elif self.activation == 'sigmoid':
            return A  * (1 - A)
        elif self.activation == 'tanh':
            return 1 - A ** 2
        else:
            return A
    
    def __repr__(self):
        return f"weigths: {self.weights.shape}\n bias: {self.bias.shape}"

In [ ]:
class DeepNeuralNetwork:
    def __init__(self, lambda_=1e-3, lr=1e-3, max_epochs=10000, 
                 batch_size=64, optimizer='adam', beta1=0.9,
                 beta2=0.999, epsilon=1e-8, verbose=1, random_state=0):
        self.layers = []
        self.lr = lr
        self.lambda_ = lambda_
        self.max_epochs = max_epochs
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.beta1 = beta1
        self.beta2 = beta2
        self.verbose = verbose
        self.epsilon = epsilon
        self.seed = random_state
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def feed_forward(self, X):
        for layer in self.layers:
            X = layer.activate(X)
        return X

    def loss(self, y_true, y_preds):
        # 交叉熵误差
        assert(y_true.shape == y_preds.shape)
        term = 0
        for layer in self.layers:
            term += np.sum(layer.weights ** 2)
        loss_ = - y_true* np.log(y_preds + self.epsilon) - (1 - y_true)*np.log(1 - y_preds + self.epsilon)  
        loss_ =  np.mean(loss_, axis=0) + self.lambda_ * term

        return np.squeeze(loss_)
    
    def backpropagation(self, X, y, t):
        # 反向传播算法 计算每一层的dW, db
        # 前向计算 得到输出值
        m = X.shape[0]  # batch size
        self.feed_forward(X)  # (m, 1)
        # print(out.shape)
        # loss_ = self.loss(y, out)
        for i in reversed(range(len(self.layers))):  # 从最后一层开始
            layer = self.layers[i]
            o_i = X if i == 0 else self.layers[i-1].A
            if layer == self.layers[-1]:  # 输出层
                # 使用 交叉熵 误差
                assert(y.shape == layer.A.shape)
                layer.dA = -y/(layer.A + self.epsilon) + (1-y) / (1-layer.A + self.epsilon)
                # layer.dA = -y / out + (1-y)/(1-out)  # dL/dA^K  # (m, 1)
            else:  # 隐藏层
                next_layer = self.layers[i + 1]
                layer.dA = next_layer.dZ @ next_layer.weights.T  # dL/dA^J (m, 1) (1, k)
                # dL/dZ^J (m, k) (m,k)
            layer.dZ = layer.dA * layer.derivative(layer.A)
            layer.dW =  o_i.T @ layer.dZ * 1 / m
            layer.db = np.sum(layer.dZ) * 1 / m
            w_term = self.lambda_ * layer.weights * 1 / m

            if self.optimizer == 'Momentum':
                layer.vdW = self.beta1 * layer.vdW + (1 - self.beta1) * layer.dW
                layer.vdb = self.beta1 * layer.vdb + (1 - self.beta1) * layer.db
                layer.weights -= self.lr * (layer.vdW + w_term)
                layer.bias -= self.lr * layer.vdb
            elif self.optimizer == 'RMSprop':
                layer.sdW = self.beta2 * layer.sdW + (1 - self.beta2) * layer.dW ** 2
                layer.sdb = self.beta2 * layer.sdb + (1 - self.beta2) * layer.db ** 2  
                layer.weights -= self.lr * (layer.dW / np.sqrt(layer.sdW + self.epsilon) + w_term)
                layer.bias -= self.lr * layer.db / np.sqrt(layer.sdb + self.epsilon)
            elif self.optimizer == 'Adam':
                vdW = self.beta1 * layer.vdW + (1 - self.beta1) * layer.dW
                vdb = self.beta1 * layer.vdb + (1 - self.beta1) * layer.db
                layer.vdW = vdW / (1 - self.beta1 ** t)
                layer.vdb = vdb / (1 - self.beta1 ** t)
                # assert(np.all(layer.vdW != np.inf))
                sdW = self.beta2 * layer.sdW + (1 - self.beta2) * layer.dW ** 2
                sdb = self.beta2 * layer.sdb + (1 - self.beta2) * layer.db ** 2
                layer.sdW = sdW / (1 - np.power(self.beta2, t))
                layer.sdb = sdb / (1 -  np.power(self.beta2, t))
                # print(layer.sdW)
                # assert(np.all(layer.sdW != np.inf))
                layer.weights -= self.lr * layer.vdW / np.sqrt(layer.sdW + self.epsilon) + self.lr * w_term
                layer.bias -= self.lr * layer.vdb / np.sqrt(layer.sdb + self.epsilon)                       
            else:
                layer.weights -= self.lr * (layer.dW + w_term)
                layer.bias -=  self.lr * layer.db
         
    def preprocessing(self, X, y):
        y = y.reshape(-1, 1)
        self.seed += 1
        np.random.seed(self.seed)
        permutation = np.random.permutation(X.shape[0])
        shuffled_X = X[permutation]
        shuffled_y = y[permutation]
        
        return shuffled_X, shuffled_y
    
    def fit(self, X, y):
        # X, y = self.preprocessing(X, y)
        cross_entropy = []
        # accuracy = []
        batch_size = self.batch_size
        m = X.shape[0]
        batches = m // batch_size if m % batch_size == 0 else m // batch_size + 1
        
        for epoch in range(self.max_epochs):
            # 每个epoch X,y 顺序都不一样
            t = 0
            shuffled_X, shuffled_y = self.preprocessing(X, y)
            # print(self.seed)
            for step in range(batches):
                
                batch_X = shuffled_X[batch_size*step: batch_size*(step+1)]
                batch_y = shuffled_y[batch_size*step: batch_size*(step+1)]
                # t 需要从1开始
                t += 1
                self.backpropagation(batch_X, batch_y, t)
            if epoch % 100 == 0:
                loss = self.loss(y, self.feed_forward(X))
                cross_entropy.append(loss)
            if epoch % 1000 == 0 and self.verbose:
#                 predict = self.predict(X_train)
#                 train_acc = np.sum(y_train.ravel()==predict) / len(predict)
                
#                 predict = self.predict(X_test)
#                 acc = np.sum(y_test.ravel()==predict) / len(predict)
#                 accuracy.append(acc)
                print(f'Epoch: {epoch}, cross_entropy: {loss}')
        print(t)
        return cross_entropy
    
    def predict(self, X):
        y_pred = self.feed_forward(X)
        y_pred = np.where(y_pred >=0.5, 1, 0)
        # out = np.argmax(y_pred, axis=1)
        return np.squeeze(y_pred)

In [ ]:
def plot_decision(model, X, y):
    x_max = X[:, 0].max() + 1
    x_min = X[:, 0].min() - 1
    y_max = X[:, 1].max() + 1
    y_min = X[:, 1].min() - 1
    xx, yy = np.mgrid[x_min:x_max:0.01, y_min:y_max:0.01]
    zz = np.c_[xx.ravel(), yy.ravel()]
    preds = model.predict(zz)
    plt.contourf(xx, yy, preds.reshape(xx.shape), cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[:, 0], X[:, 1], c=np.squeeze(y), cmap=plt.cm.Spectral)
    plt.show()